# Dynamic Programming
This chapter is a bit of an oddity to me because I have always been taught to think of dynamic programming as a matter of decomposition and memoization, which it turns out does play into the history of optimal control...  The thing that throws me for a loop here is that I usually think of dynamic programming as an algorithmic strategy that you apply for some problems with the right structure.  I guess the structure is simply that you can express the optimal solution recursively, in terms of the optimal solution to a smaller problem, and that is certainly the gist of the Bellman equations, so there's that... 

Anyhow, this chapter starts to get into how to find optimal policies for _finite MDPs_ (i.e., finite state, reward, and action spaces) when we have a good model of the dynamics $p(s', r | s, a)$.  The basic idea is to use the Bellman optimality equations.  As a refresher, here they are again: 

$$
\begin{align}
v_*(s) & = \max_a \mathbb{E}[R_{t+1} + \gamma v_*(S_{t+1}) | S_t = s, A_t = a ] \\
       & = \max_a \sum_{s', r} p(s', r|s,a) [r + \gamma v_*(s)] \\
q_*(s,a) & = \mathbb{E} \left[ R_{t+1} + \gamma max_{a'} q_*(S_{t+1}, a' | S_t = s, A_t = a \right] \\       
         & = \sum_{s', r} p(s', r|s,a) [r + \gamma max_{a'} q_*(s,a')]
\end{align}
$$

## Iterative Policy Evaluation
First off, it seems reasonable that in order to improve on a given policy, $\pi$, we need a way to evaluate how good it is.  This is called _policy evaluation_, or the _prediction problem_.  Note that we are not claiming that $\pi$ is optimal - we are just asking - how do we evaluate the value of an arbitrary $\pi$?  Recall one form of the Bellman equation for state value functions: 

$$
v_{\pi}(s) = \sum_a \pi(a|s) \sum_{s', r} p(s',r|s,a) [r + \gamma v_{\pi}(s')]
$$

First off, we note that $v_{\pi}$ exists and is unique as long as $\gamma < 1$ or termination is guaranteed from all states under the policy.  In principle, the above relation defines a linear system of equations we could solve but that is tedious and in any event this kind of approach simply won't scale for bigger problems.  So we focus on iterative approaches.  We start with a initial guess at $v_{\pi}, v_0$, and we improve on it (make it closer to the actual value function) iteratively using the update: 

For each state:
$$
\begin{align}
v_{k+1}(s) & = \mathbb{E}_{\pi} [ R_{t+1} + \gamma v_k(S_{t+1}) | S_t = s ] \\
           & = \sum_a \pi(a|s) \sum_{s',r} p(s',r|s,a) [r + \gamma v_k(s)] \\
\end{align}
$$

That is, we simply use the Bellman equation as an update.  Under the same conditions that guarantee the existence and uniqueness of $v_{\pi}$, we can prove that the sequence of $v_k$ converges to $v_{\pi}$ - we can definitely see that the latter is a fixed point of these updates, so this certainly seems reasonable!  Note - these kinds of updates are called _expected updates_ because we are calculating them using the expectation of the next state and reward rather than a single experiment or sample of the next state and reward...  

### How we do this in practice
First, analogous to Gibbs sampling, this is most often done _in place_.  Say we are representing $v_k$ as an array with an element for each state $s$.  The most straight forward way to run this is to start with such a vector for iteration $k$, and then for iteration $k+1$ we write into a new array representing $v_{k+1}$.  In practice, however, we converge faster if we keep a single array (i.e., we update value in place) so that later updates get the benefit of the improved estimates for earlier states.  We also need some way to decide when we've effectively converged - we can just test whether the maximum of $\max_s v_{k+1}(s) - v_k(s) < \epsilon$  for some threshold small $\epsilon$.  If it is, we stop.
           

<b>Exercise 4.3</b>  What are the analogous equations for the action-value function $q_{\pi}$?

For each state and action pair, we update:
$$
q_{k+1}(s,a) = \sum_{s',r} p(s',r|s,a) [ r + \gamma max_{a'} q_k(s,a') ]
$$

## Policy Improvement
So now that we can evaluate a policy, how do we actually improve on a given policy $\pi$?  Specifically, say we are considering changing $\pi$ to deterministically select action $a$ in state $s$.  The value of doing this is given simply by the current action-value function (we have used policy evaluation to get $v_{\pi}$): 

$$
q_{\pi}(s,a) = \sum_{s',r} p(s',r|s,a) [r + \gamma v_{\pi}(s')]
$$

If this is greater than $v_{\pi}(s)$, then, because of the Markov property, it is reasonable to assume that we should do action $a$ every time we are in state $s$, so we should update $\pi$ to do $a$ when in $s$.  This is the basic idea behind the _policy improvement theorem): let $\pi$ and $\pi'$ be any pair of deterministic policies, with: 

$$
q_{\pi}(s, \pi'(s)) \ge v_{\pi}(s)
$$

This is just saying the same thing as the above, but jumping straight to the relevant comparison.  Then $\pi'$ is as good as or better than $\pi$: 

$$
v_{\pi'}(s) \ge v_{\pi}(s) \text{  } \forall s \in \mathcal{S}
$$

The proof of this is simple - see page 78.  

So what happens if we expand this to consider changes to all states and all possible actions?  That is, for each state, update $\pi \rightarrow \pi'$ to select the action $a$ in a greedy fashion? 

For each state, we update the policy:
$$
\begin{align}
\pi(s) & = \underset{a}{argmax} q_{\pi}(s,a) \\
       & = \underset{a}{argmax} \mathbb{E}[R_{t+1} + \gamma v_{\pi}(S_{t+1}) | S_t = s, A_t = a ] \\
       & = \underset{a}{argmax} \sum_{s',r} p(s',r|s,a) [r + \gamma v_{\pi}(s)]
\end{align}
$$

## Policy Iteration
Now let's say that we do alternating rounds of policy evaluation and improvement until we are no longer making any updates.  Then we know that the Bellman optimality condition holds, since it certainly holds for any given state by construction.  Thus, when we have converged, we should have an optimal value function and policy!  This algorithm is called _policy iteration_.

When I first read this, I was a bit confused - won't this immediately give you the optimal policy?  But no, it won't - recall that this procedure is for a given policy $\pi$, and we are updating that each time we update $\pi$ (we have to do another round of policy evaluation for each iteration of policy improvement).  The process looks like this.  Say we start with an initial policy $\pi_0$...  We do iterations of policy evaluation and improvement: 

$$
\pi_0 \overset{E}{\rightarrow} v_{\pi_0} \overset{I}{\rightarrow} \pi_1 \overset{E}{\rightarrow} v_{\pi_1} \overset{I}{\rightarrow} \pi_2 \overset{E}{\rightarrow} v_{\pi_2} \overset{I}{\rightarrow} \pi_3 \dots \overset{I}{\rightarrow} \pi_* \overset{E}{\rightarrow} v_{\pi_*}
$$

After each round of policy improvement, we've improved the policy but still only with respect to the current policy - we don't necessarily have the optimal policy!  So we have to keep going until we are no longer updating the value function or policy, in which case we have converged per the discussion above.

Finally, note that this was all written with respect to deterministic policies, but in fact holds for stochastic policies as well - just assign zero probability to all non-optimal actions in the update. Each round of policy iteration improves on the previous policy, and since we have a finite MDP this will eventually converge.

## Exercises 4.4-7
These are worth doing...

## Value Iteration
The above algorithm must do policy evaluation every iteration, which is itself a potentially quite expensive iterative process.  It turns out that we can just truncate it after one iteration and combine the update with policy improvement - the resulting algorithm is called _value iteration_:

For each state, we update: 
$$
v_{k+1}(s) = \max_a \sum_{s',r} p(s',r|s,a)[ r + \gamma v_k(s') ]
$$

Note that unlike the policy evaluation iterations, we are not doing an expectation over actions under policy $\pi$ - instead we are taking the max, effectively updating $v$ while also trying to improve the policy.  This converges to $v_*$ under the same conditions that guarantee the existence of $v_*$.  As for policy evaluation, we stop when $v$ changes very little over all states.

### Exercises 4.8-9

## Asynchronous Dynamic Programming
Each iteration of policy evaluation and policy improvement as described above requires a full sweep through all states $s \in \mathcal{S}$, which can be very expensive.  It often works well to update values of for only some subset of states per iteration.  Indeed, some states may have their values updated multiple times before others are updated once.  However, in order to converge, we must eventually get around to updating all states of course.  At an extreme, we can update the value of only a single state at a time!  This sort of game is called asynchronous dynamic programming, and is very useful for online learning situations where we are in a single state IRL!

## Generalized Policy Iteration
There is a lot of room between our two extremes of iterations of policy-evalution followed policy improvement vs value iteration.  For instance, one can often get fast convergence by doing multiple rounds of policy evaluation for each policy improvement sweep.  GPI is an umbrella term for algorithms that alternate policy evaluation (not always full - see value iteration!) and policy improvement, and includes almost all RL algorithms in practice.  